In [10]:
#KRYSTIAN MOWINSKI 
#MOW21509673
#
import math
import random
from PIL import Image
import os
from time import sleep
from IPython.display import clear_output

city_coords = { # coordinates of all locations to get relative heuristic distances
    "manchester": (53.4808, -2.2426),
    "holyhead": (53.3071, -4.6318),
    "liverpool": (53.4084, -2.9916),
    "york": (53.9590, -1.0817),
    "carlisle": (54.8924, -2.9324),
    "newcastle": (54.9783, -1.6178),
    "glasgow": (55.8642, -4.2518),
    "edinburgh": (55.9533, -3.1883),
    "oban": (56.4127, -5.4702),
    "aberdeen": (57.1497, -2.0943),
    "inverness": (57.4778, -4.2247)
}

city_distances = {#city distances for other searching algorithms
      "manchester": {"liverpool": 54, "york": 113, "newcastle": 235, "edinburgh": 354, "carlisle": 196},
      "holyhead": {"liverpool": 150},
      "liverpool": {"manchester": 55, "holyhead": 150},
      "york": {"manchester": 112, "newcastle": 136},
      "carlisle": {"manchester": 195, "glasgow":154},
      "newcastle": {"manchester": 231,"york": 136, "edinburgh": 167},
      "glasgow": {"edinburgh": 75,"carlisle": 154,"oban": 155,"inverness": 273,"aberdeen": 233,},
      "edinburgh": {"manchester": 344,"newcastle": 166,"glasgow": 74,},
      "oban": {"glasgow": 154, "inverness": 172},
      "aberdeen": {"inverness": 167,"glasgow": 230},
      "inverness": {"glasgow": 271, "oban": 174, "aberdeen": 167}
  }

heuristic_distances = {#base for heuristic distances
    "manchester": 0,
    "holyhead": 0,
    "liverpool":0 ,
    "york": 0,
    "carlisle": 0,
    "newcastle": 0,
    "glasgow": 0,
    "edinburgh": 0,
    "oban": 0,
    "aberdeen": 0,
    "inverness": 0
  }

def runalgorithms(start, goal,speed):#running all search algorithms
  print("================================================================================")
  depth_path, depth_Distance = depth_first_search(start, goal)#getting distance and path
  #print(depth_path, depth_Distance)
  print(depth_Distance,"km, is the distance of the entire path")
  depth_traffic = traffic_level()
  depth_time = get_time(speed, depth_Distance, depth_traffic)
  for node in depth_path:
        print(node, end=" ")
        if node != goal:
            print("--->", end=" ")
  print()
  print("================================================================================")
  breadth_path, breadth_Distance = breadth_first_search(start, goal)
  #print(breadth_path,breadth_Distance)
  print(breadth_Distance,"km, is the distance of the entire path")
  breadth_traffic = traffic_level()
  breadth_time = get_time(speed, breadth_Distance, breadth_traffic)
  for node in breadth_path:
        print(node, end=" ")
        if node != goal:
            print("--->", end=" ")
  print()
  print("================================================================================")
  for city in heuristic_distances:
      if city == start:
          heuristic_distances[city] = 0  # the heuristic distance from the start city to itself is zero
      else:
          city_coords1 = city_coords[start]
          city_coords2 = city_coords[city]
          distance = haversine(city_coords1[0], city_coords1[1], city_coords2[0], city_coords2[1])
          heuristic_distances[city] = distance
    #print(heuristic_distances)
  A_path, A_distance = a_star_search(start,goal,heuristic_distances)
  #print(A_path,A_distance)
  print(A_distance,"km, is the total distance of the enitre path")
  A_traffic = traffic_level()
  A_time = get_time(speed,A_distance,A_traffic)
  for node in A_path:
        print(node, end=" ")
        if node != goal:
            print("--->", end=" ")
  print()
  print("================================================================================")
  Dijkstras_path, Dijkstras_distance = dijkstra_search(start, goal)
  #print(Dijkstras_path,Dijkstras_distance)
  print(Dijkstras_distance,"km, is the total distance of the entire path")
  Dijkstras_traffic = traffic_level()
  Dijkstras_time = get_time(speed, Dijkstras_distance, Dijkstras_traffic)
  for node in Dijkstras_path:
        print(node, end=" ")
        if node != goal:
            print("--->", end=" ")
  print()
  print("================================================================================")


def print_image():#printing an image 
  print("here is the map of places you can travel to: ")
  image = Image.open("/content/pciture school work.png")
  width, height = image.size
  new_width = 350#making image smaller
  new_height = int(height * (new_width / width))#keeping the same aspect ratio for the image
  resized_image = image.resize((new_width, new_height))#resized image saved

  #Display the resized image
  resized_image.show()#showing the image

def haversine(lat1, lon1, lat2, lon2):#getting the straight line distances between 2 places (really confusing)
  R = 6371  # radius of the Earth in kilometers
  phi1 = math.radians(lat1)
  phi2 = math.radians(lat2)
  delta_phi = math.radians(lat2 - lat1)
  delta_lambda = math.radians(lon2 - lon1)
  a = math.sin(delta_phi / 2)**2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
  distance = R * c
  return distance


def get_time(speed, distance, extra_time):#getting the time it would take to drive at a distance given by the user 
  time = distance/speed
  time = time + extra_time #adding extra time to these because of the traffic implementation
  print(time , "Hours Is the amount of time it will take you to get here")# printing a message to inform the user


def get_speed(): #asking the user for the speed
    while True:
        try:
            speed = int(input("Enter your driving speed in km/h (0-110): "))
            if 0 <= speed <= 110:#making sure speed is between the limits 
                return speed
            else:
                print("Speed must be between 0 and 110.")#incase there is an error
        except ValueError:
            print("Invalid input. Please enter a number.")#incase there is an invalid input
    

def traffic_level():#randomising traffic (to simulate real life experiences)
  Traffic_levels = ["None","Low","Moderate", "High"]
  Traffic = random.choice(Traffic_levels)#randomising
  if Traffic == "None":
    print("There is no traffic today (This will add nothing)")
    extra_time = 0 #assigning values to the extra time in hours 
  elif Traffic == "Low":
    print("todays traffic is Low (This will ad an extra half hour to your journey)")
    extra_time = 0.5 #half an hour
  elif Traffic == "Moderate":
    print("Todays traffic is moderate (This will ad an extra hour to your journey)")
    extra_time  = 1 #1 hour
  elif Traffic == "High":
    print("Todays traffic is High (This will ad an extra 2 hours to your journey)")
    extra_time = 2 #2 hours
  else: 
    print("this is not an option")

  return extra_time


# Define the search algorithms
def depth_first_search(start, goal, visited=None, total_distance=0): #depth first search
    if visited is None:
        visited = set()
    visited.add(start)#adding to list 
    if start == goal:
        return [start], total_distance #if the goal is the start return the values
    for next_city in city_distances[start]:
        if next_city not in visited:# making sure we havent been to that city already so we dont go back
            distance = city_distances[start][next_city]#calulating distance
            path, distance_to_goal = depth_first_search(next_city, goal, visited, total_distance + distance)
            if path:
                return [start] + path, distance_to_goal
    return None, 0 #if nothing is found



def breadth_first_search(start, goal):#breadth first search
    queue = [(start, [start], 0)] 
    while queue:
        (current_city, path, distance_traveled) = queue.pop(0)
        if current_city == goal:
            return (path, distance_traveled) 
        for next_city in city_distances[current_city]:
            if next_city not in path:
                distance_to_next_city = city_distances[current_city][next_city]
                queue.append((next_city, path + [next_city], distance_traveled + distance_to_next_city))
    return None


def a_star_search(start, goal, heuristic_distances): # a star algorithm
    queue = [(heuristic_distances[start], 0, start, [])]
    visited = set()
    while queue:
        _, distance, current_city, path = queue.pop(0)
        if current_city == goal:
            return path + [current_city], distance
        visited.add(current_city)
        for next_city in city_distances[current_city]:
            if next_city not in visited:
                new_distance = distance + city_distances[current_city][next_city]
                heuristic_distance = heuristic_distances[next_city]
                queue.append((new_distance + heuristic_distance, new_distance, next_city, path + [current_city]))
        queue.sort()
    return None

def dijkstra_search(start, goal): # dijkstras algorithm
    queue = [(0, start, [])]
    visited = set()
    while queue:
        (distance, current_city, path) = queue.pop(0)
        if current_city == goal:
            return path + [current_city], distance
        visited.add(current_city)
        for next_city in city_distances[current_city]:
            if next_city not in visited:
                total_distance = distance + city_distances[current_city][next_city]
                queue.append((total_distance, next_city, path + [current_city]))
                queue.sort()
    return [], float('inf')

def main():#defining main
  print_image()#running print image to show image 

  sleep(5)#waiting 5 seconds

  clear_output()#clearing the console 

  cities =["Manchester","Liverpool","Holyhead","York","Newcastle","Edinburgh","Glasgow","Oban","Inverness","Aberdeen","Carlisle"] #cities possible
  print("These are the cities available:")
  for i in range(len(cities)):
    print(cities[i])
    cities[i] = cities[i].lower() # making them all lower case
  
  start = None #none type
  while start not in cities:# making sure an input stays within range of my list of cities
    try:
      start = input("Please enter the place you are starting at: ").lower() #making it lower case so that the inputs are not case sensitive this allows for mis spelling
    except:
      print("Please enter one of the cities provided")

  goal = None # doing same thing but to the goal
  while goal not in cities:
    try:
      goal = input("Please enter your end destination: ").lower()
    except:
      print("please enter your end destination that is in the list of cities")
  
  speed = get_speed()# getting the speed 
  #print(depth_first_search(start,goal))
  #print(breadth_first_search(start,goal))
  #print("")

  sleep(2)#sleep for 2 seconds
  clear_output()#clearing the python console


  print("1)depth first search. \n2)Breadth first search. \n3)A star algorithm. \n4)dijkstras Search. \n5)All searches.") #showing user all choices
  choice1 = None
  while choice1 not in [1,2,3,4,5]:# making sure choice is in bounds
    try:
      choice1 = int(input(" :"))#making sure input is an integer
    except:
      print("Please enter a number from 1 to 5")#exception if input is not an integer
  
  if choice1 == 1 :#choice 1 
    sleep(2)
    clear_output()
    print("You have picked Depth first search")
    path, distance = depth_first_search(start, goal)
    print(distance,"KM, is the whole distance of the journey")
    #print(path,Distance)
    traffic = traffic_level()
    time = get_time(speed,distance,traffic)
    for node in path:
        print(node, end=" ")
        if node != goal:
            print("--->", end=" ")
    print()
    
  elif choice1 == 2:#choice 2
    sleep(2)
    clear_output()
    print("You have picked Breadth first search")
    path, distance = breadth_first_search(start, goal)
    print(distance,"KM, is the whole distance of the journey")
    #print(path,Distance)
    traffic = traffic_level()
    time = get_time(speed,distance,traffic)
    for node in path:
        print(node, end=" ")
        if node != goal:
            print("--->", end=" ")
    print()

  elif choice1 == 3: #choice 3
    sleep(2)
    clear_output()
    print("You have picked A star Algorithm")
    for city in heuristic_distances:
      if city == start:
          heuristic_distances[city] = 0  # the heuristic distance from the start city to itself is zero
      else:
          city_coords1 = city_coords[start]
          city_coords2 = city_coords[city]
          distance = haversine(city_coords1[0], city_coords1[1], city_coords2[0], city_coords2[1])
          heuristic_distances[city] = distance
    #print(heuristic_distances)
    path, distance = a_star_search(start,goal, heuristic_distances)
    #print(path, distance)
    print(distance,"KM, is the whole distance of the journey")
    traffic = traffic_level()
    time = get_time(speed,distance,traffic)
    for node in path:
        print(node, end=" ")
        if node != goal:
            print("--->", end=" ")
    print()

  elif choice1 == 4: #choice 4
    sleep(2)
    clear_output()
    print("You have chosen Dijkstra's Algorithm")
    path, distance = dijkstra_search(start, goal)
    #print(path, distance)
    print(distance,"KM, is the whole distance of the journey")
    traffic = traffic_level()
    time = get_time(speed,distance,traffic)
    for node in path:
        print(node, end=" ")
        if node != goal:
            print("--->", end=" ")
    print()

  elif choice1 == 5:# choice 5
    sleep(2)
    clear_output()
    runalgorithms(start,goal,speed)
    

while True:#repeating the main program
    try:
        main()
        #print("output will be cleared in 15 seconds")
        #sleep (15)
        #clear_output()
        
    except:
        print("There has been an error.")#if there is an error with the program print this message
    finally:#finally when everything else has been run
        while True: #asking if the user wants to do all of this
            again = input("Do you wish to continue? y/n : ").lower() #making sure its not case sensitive
            if again in ["n", "no"]: # defining range of inputs
                print("Thank you for using our program!")
                break#stopping this while loop
            elif again in ["y", "yes"]: #stopping this and running main because of the next if
                break#stopping this loop
            else:#else please enter a good input
                print("Invalid input. Please enter y or n.")
        if again in ["n", "no"]:# input ranges so that it stops the next loop if input is no
            break# breaking the second loop and stopping the program



308 km, is the distance of the entire path
Todays traffic is High (This will ad an extra 2 hours to your journey)
17.4 Hours Is the amount of time it will take you to get here
carlisle ---> manchester ---> york 
308 km, is the distance of the entire path
todays traffic is Low (This will ad an extra half hour to your journey)
15.9 Hours Is the amount of time it will take you to get here
carlisle ---> manchester ---> york 
308 km, is the total distance of the enitre path
todays traffic is Low (This will ad an extra half hour to your journey)
15.9 Hours Is the amount of time it will take you to get here
carlisle ---> manchester ---> york 
308 km, is the total distance of the entire path
Todays traffic is High (This will ad an extra 2 hours to your journey)
17.4 Hours Is the amount of time it will take you to get here
carlisle ---> manchester ---> york 
Do you wish to continue? y/n : n
Thank you for using our program!
